<a href="https://colab.research.google.com/github/ttogle918/AI_practice/blob/main/QA%20task/02_BERT_QA_squad_BertForQuestionAnswering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

해야할 것!

1. 평가지표 바꾸기
2. train에서 answer과 비교하기! 수식 작성

In [ ]:
!pip install transformers
!pip install datasets

In [20]:
from datasets import load_dataset, load_metric
from transformers import BertForQuestionAnswering, AutoTokenizer
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import f1_score
from scipy import stats
import time
import matplotlib.pyplot as plt

In [ ]:
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

# Data Load

In [7]:
squad = load_dataset("squad")
squad, squad['train'][0]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

(DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
}),
 {'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
  'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the

id로 중복값 제거해야할까

In [ ]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.context, self.question, self.answers = self.make_dataset(dataset)

    def make_dataset(self, dataset):
        context, question, answers = [], [], []
        global max_length
        for i, data in enumerate(dataset) :
          answer_start, text = data['answers']['answer_start'], data['answers']['text']
          if len(answer_start) != 1 :
            print(i, data)
            continue
          answers.append([answer_start[0], answer_start[0] + len(text)])  # 정답의 시작과 끝 index
          context.append(data['context'])
          question.append(data['question'])
        return context, question, answers
        
    def __len__(self):
        return len(self.question)

    def __getitem__(self, idx):
        return self.context[idx], self.question[idx], self.answers[idx]

In [ ]:
max_length = 0
def custom_collate_fn(batch):
    global max_length
    input1_list, input2_list, target_list = [], [], []

    for _input1, _input2, _target in batch:
        input1_list.append(_input1)
        input2_list.append(_input2)
        target_list.append(_target)
    
    tensorized_input = tokenizer( # max length 구하기! dim=1
        input1_list, input2_list,
        add_special_tokens=True,
        padding="longest",  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
        return_tensors='pt'
    )
    tensorized_label = torch.tensor(target_list)
    if max_length < len(tensorized_input['input_ids']) :    # squad 데이터에서 token의 최대 길이를 알게되면 삭제!
      max_length = len(tensorized_input['input_ids'])
      print(max_length)
    return tensorized_input, tensorized_label

In [ ]:
def make_dataloader(dataset, tokenizer, batch_size, s='train') :
  dataloader = DataLoader(
      dataset,
      batch_size =batch_size,
      sampler = RandomSampler(dataset) if s == 'train' else SequentialSampler(dataset),
      collate_fn = custom_collate_fn
  )
  print(f'batch_size : {batch_size}')
  return dataloader

# 모델 설명

['distilbert-base-uncased'](https://huggingface.co/distilbert-base-uncased) : DistilBERT는 BERT 기본 모델을 교사로 사용하여 자체 지도 방식으로 동일한 코퍼스에서 사전 훈련된 BERT보다 작고 빠른 변환기 모델

In [ ]:
class CustomBertForQuestionAnswering(nn.Module):
    def __init__(self, model_name):
        super(self).__init__()
        self.model = BertForQuestionAnswering.from_pretrained(model_name)

    def forward(self, token_ids, valid_length, segment_ids):
        return self.model(token_ids, token_type_ids = segment_ids)

In [8]:
model = BertForQuestionAnswering.from_pretrained('distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForQuestionAnswering: ['distilbert.transformer.layer.1.sa_layer_norm.bias', 'distilbert.transformer.layer.3.attention.v_lin.bias', 'distilbert.transformer.layer.4.output_layer_norm.bias', 'distilbert.transformer.layer.3.ffn.lin2.weight', 'distilbert.transformer.layer.5.sa_layer_norm.bias', 'distilbert.transformer.layer.2.output_layer_norm.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.transformer.layer.2.attention.q_lin.bias', 'distilbert.transformer.layer.1.attention.q_lin.weight', 'distilbert.transformer.layer.4.sa_layer_norm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.2.attention.q_lin.weight', 'vocab_layer_norm.weight', 'distilbert.transformer.layer.4.attention.k_lin.weight', 'distilbert.transformer.layer.4.ffn.lin2.weight', 'distilbert.transformer.layer.0.ffn.lin2.bias', 'distilbert.transformer.layer.5.ffn.lin1.weight', 'v

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

# train

In [ ]:
def initializer(train_dataloader, epochs=2, model_name='snunlp/KR-Medium', lr=4e-5, wd=4e-5):
    """
    모델, 옵티마이저, 스케쥴러 초기화
    """
    model = CustomBertForQuestionAnswering(model_name=model_name)

    optimizer = AdamW(
        model.parameters(), # update 대상 파라미터를 입력
        lr=lr,    # 2e-5
        eps=1e-8,
        weight_decay=wd
    )
    
    total_steps = len(train_dataloader) * epochs
    print(f"Total train steps with {epochs} epochs: {total_steps}")

    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps = 0, # 여기서는 warmup을 사용하지 않는다.
        num_training_steps = total_steps
    )
    print(f'model_name : {model_name}, lr : {lr}, weight_decay : {wd}, epochs : {epochs}')
    return model, optimizer, scheduler

In [ ]:
def save_checkpoint(path, model, optimizer, scheduler, epoch, loss, f1, model_name=''):
    file_name = f'{path}/epoch:{epoch}_loss:{loss:.4f}_f1:{f1:.4f}.ckpt'
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss' : loss,
            'f1' : f1
        }, 
        file_name
    )
    
    print(f"Saving epoch {epoch} checkpoint at {file_name}")

### train code

평가지표 확인!

In [ ]:
before_loss, before_f1 = 0.4, 0.7   # 이 값보다 좋지 않으면 checkpoint 생성하지 않기 위해서 설정

In [ ]:
def train(model, optimizer, scheduler, train_dataloader, valid_dataloader=None, epochs=1, model_name=''):
  loss_fct = nn.MSELoss()
  train_dict = {'loss' : [], 'f1' : []}
  valid_dict = {'loss' : [], 'f1' : [], 'pearson' : []}
  global before_loss, before_f1
  for epoch in range(epochs) :

    print(f"*****Epoch {epoch} Train Start*****")
    total_loss, total_f1, batch_f1, batch_loss, batch_count = 0,0,0,0,0
    
    model.train()
    model.to(device)
    
    for step, batch in enumerate(train_dataloader):
      batch_count+=1
      
      batch = tuple(item.to(device) for item in batch)
      batch_input, batch_label = batch
      
      model.zero_grad()
      
      probs = model(**batch_input)  # forward

      loss = loss_fct(probs, batch_label)
      batch_loss += loss.item()
      total_loss += loss.item()

      f1 = f1_score([0 if p < 3 else 1 for p in batch_label], [0 if p < 3 else 1 for p in probs])
      batch_f1 += f1
      total_f1 += f1

      # backward -> 파라미터의 미분(gradient)를 자동으로 계산
      loss.backward()

      # gradient clipping 적용 
      clip_grad_norm_(model.parameters(), 1.0)
      
      # optimizer & scheduler 업데이트
      optimizer.step()
      scheduler.step()

      # 그래디언트 초기화
      model.zero_grad()

      if (step % 128 == 0 and step != 0):
          learning_rate = optimizer.param_groups[0]['lr']
          print(f"Epoch: {epoch}, Step : {step}, LR : {learning_rate:.10f}, Avg Loss : {batch_loss / batch_count:.4f}, f1 score : {batch_f1 / batch_count:.4f}")
          
          if (round(batch_f1 / batch_count, 5) == 0) and (round(learning_rate, 10) == 0) :
              print("Train Finished, learning_rate is 0 and train_f1 is 0")
              return train_dict, valid_dict

          batch_loss, batch_f1, batch_count = 0,0,0


    print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
    print(f"Epoch {epoch} Total Mean f1 : {total_f1/(step+1):.4f}")
    print(f"*****Epoch {epoch} Train Finish*****\n")

    train_dict['f1'].append(total_f1/(step+1))
    train_dict['loss'].append(total_loss/(step+1))
    
    if valid_dataloader is not None:
        print(f"*****Epoch {epoch} Valid Start*****")
        valid_loss, valid_acc, valid_f1, valid_pearson = validate(model, valid_dataloader)
        print(f"Epoch {epoch} Valid Loss : {valid_loss:.4f} Valid Acc : {valid_acc:.4f} Valid f1 : {valid_f1:.4f}")
        print(f"pearson 상관 계수 ; {valid_pearson}")
        print(f"*****Epoch {epoch} Valid Finish*****\n")

    valid_dict['f1'].append(valid_f1)
    valid_dict['loss'].append(valid_loss)
    if round(valid_f1, 4) == 0 :
        break
    if before_loss > valid_loss :
        before_loss = valid_loss
        save_checkpoint("/content/drive/MyDrive/Colab Notebooks/nlp", model, optimizer, scheduler, epoch, valid_loss, valid_f1, model_name)

    elif before_f1 < valid_f1  :
        before_f1 = valid_f1
        save_checkpoint("/content/drive/MyDrive/Colab Notebooks/nlp", model, optimizer, scheduler, epoch, valid_loss, valid_f1, model_name)

  print("Train Finished")
  return train_dict, valid_dict

### validation code

In [ ]:
def validate(model, valid_dataloader):
    loss_fct = nn.MSELoss()

    model.eval()
    model.to(device)
    
    total_loss, total_acc, total_f1, total_pearson= 0,0, 0, 0
        
    for step, batch in enumerate(valid_dataloader):
        
        batch = tuple(item.to(device) for item in batch)
            
        batch_input, batch_label = batch
            
        # gradient 계산하지 않음
        with torch.no_grad():
            probs = model(**batch_input)
            
        loss = loss_fct(probs, batch_label)
        total_loss += loss.item()
        
        acc = 0
        for p, b in zip(probs, batch_label) :
          if (p > 3 and b > 3) or (p < 3 and b < 3 ) :
            acc += 1
        
        acc = acc / len(probs)
        total_acc+=acc
        
        pearson = torch.corrcoef(torch.stack([probs, batch_label], dim=0))
        total_pearson += pearson

        f1 = f1_score([0 if p < 3 else 1 for p in batch_label], [0 if p < 3 else 1 for p in probs])
        total_f1 += f1

    total_loss = total_loss/(step+1)
    total_acc = total_acc/(step+1)
    total_f1 = total_f1/(step+1)
    total_pearson = total_pearson/(step+1)
    return total_loss, total_acc, total_f1, total_pearson

### draw_plot

In [ ]:
# loss와 f1-score의 변화를 epoch마다 보기 위한 plot
def draw_plot(train_dict, valid_dict, i) :
  print('green is loss, gray is f1')
  plt.subplot(1, 2, 1)
  plt.xlabel('Epochs')
  plt.title('Loss and F1 of Train data')
  x_values= [n for n in range(len(train_dict['loss']))]
  plt.plot(x_values, train_dict['loss'], color='green', marker='o')  # loss
  plt.plot(x_values, train_dict['f1'], color='#AAAAAA', marker='*')  # f1

  plt.subplot(1, 2, 2)
  plt.xlabel('Epochs')
  plt.title('Loss and F1 of Validation data')
  x_values= [n for n in range(len(valid_dict['loss']))]
  plt.plot(x_values, valid_dict['loss'], color='green', marker='o')  # loss
  plt.plot(x_values, valid_dict['f1'], color='#AAAAAA', marker='*')  # f1

  plt.show()
  plt.savefig(f'figure_{i}.png')

In [ ]:
model_name = 'distilbert-base-uncased'   # 다시 설정 필요

train_dataset = CustomDataset(squad['train'])
valid_dataset = CustomDataset(squad['validation'])

train_dataloader = make_dataloader(train_dataset, model_name, 64, 'train')
valid_dataloader = make_dataloader(valid_dataset, model_name, 32, 'valid')

learning_rate = 5e-5
weight_decay = 4e-5
model, optimizer, scheduler = initializer(train_dataloader, 4, model_name, learning_rate, weight_decay)
start = time.time()
train_dict, valid_dict = train(model, optimizer, scheduler, train_dataloader, valid_dataloader, 4, model_name)
end = time.time()
print(f"time : {(end - start)//60}분 {(end - start)%60}초")

draw_plot(train_dict, valid_dict, 0)

In [ ]:
import gc
gc.collect()